In [1]:
import numpy as np
import matplotlib.pyplot as plt


Matplotlib is building the font cache; this may take a moment.


In [31]:
def init_params(layer_dims):
    np.random.seed(3)
    params = {}

    for l in range(1, len(layer_dims)):
        params['W'+str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*0.01
        params['b'+str(l)] = np.zeros((layer_dims[l], 1))

    return params

In [32]:
# Z (linear hypothesis) - Z = W*X + b , 
# W - weight matrix, b- bias vector, X- Input 

def sigmoid(Z):
    A = 1/(1+np.exp(np.dot(-1, Z)))
    # cache = (Z)

    return A

In [48]:
def forward_prop(X, params):

    A = X # input to first layer i.e. training data
    caches = []
    L = len(params)//2
    for l in range(1, L+1):
        A_prev = A

        # Linear Hypothesis
        weight = params['W'+str(l)]
        bias = params['b'+str(l)]
        Z = np.dot(weight, A_prev) + bias

        # Storing the linear cache
        linear_cache = (A_prev, weight, bias)

        # Applying sigmoid on linear hypothesis
        A = sigmoid(Z)
        activation_cache = (Z)

         # storing the both linear and activation cache
        cache = (linear_cache, activation_cache)
        caches.append(cache)

    return A, caches

In [50]:
def cost_function(A, Y):
    m = Y.shape[1]

    cost = (-1/m)*(np.dot(np.log(A), Y.T) + np.dot(np.log(1-A), 1-Y.T))

    return cost

In [5]:
def one_layer_backward(dA, cache):
    linear_cache, activation_cache = cache

    Z = activation_cache
    dZ = dA*sigmoid(Z)*(1-sigmoid(Z)) # The derivative of the sigmoid function

    A_prev, W, b = linear_cache
    m = A_prev.shape[1]

    dW = (1/m)*np.dot(dZ, A_prev.T)
    db = (1/m)*np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db

In [6]:
def backprop(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)

    dAL = -(np.divide(Y, AL) - np.divide(1-Y, 1-AL))

    current_cache = caches[L-1]
    grads['dA'+str(L-1)], grads['dW'+str(L-1)], grads['db'+str(L-1)] = one_layer_backward(dAL, current_cache)

    for l in reversed(range(L-1)):

        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = one_layer_backward(grads["dA" + str(l+1)], current_cache)
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads


In [52]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2

    for l in range(1, L):
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] - learning_rate*grads['W'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] -  learning_rate*grads['b'+str(l+1)]

    return parameters

In [53]:
def train(X, Y, layer_dims, epochs, lr):
    params = init_params(layer_dims)
    print("params init-ed")
    cost_history = []

    for i in range(epochs):
        print(f'epoch: {i}')
        Y_hat, caches = forward_prop(X, params)
        print(f'epoch {i} forward')
        cost = cost_function(Y_hat, Y)
        cost_history.append(cost)
        print(f'epoch {i} cost')
        grads = backprop(Y_hat, Y, caches)
        print(f'epoch {i} back')

        params = update_parameters(params, grads, lr)
        print(f'epoch {i} udpate')


    return params, cost_history


In [10]:
import pandas as pd
chicago_taxi_dataset = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/chicago_taxi_train.csv")

In [11]:
chicago_taxi_dataset.columns

Index(['TRIP_START_TIMESTAMP', 'TRIP_END_TIMESTAMP', 'TRIP_START_HOUR',
       'TRIP_SECONDS', 'TRIP_MILES', 'TRIP_SPEED', 'PICKUP_CENSUS_TRACT',
       'DROPOFF_CENSUS_TRACT', 'PICKUP_COMMUNITY_AREA',
       'DROPOFF_COMMUNITY_AREA', 'FARE', 'TIPS', 'TIP_RATE', 'TOLLS', 'EXTRAS',
       'TRIP_TOTAL', 'PAYMENT_TYPE', 'COMPANY'],
      dtype='object')

In [54]:
input = chicago_taxi_dataset.iloc[:, 5:6].values
Y = chicago_taxi_dataset.iloc[:, 11:12].values
layer_dims = [len(input),len(input),len(input)]

output = train(input, Y, layer_dims, 5, 0.01)
output

params init-ed
epoch: 0


KeyError: 'W2'